# Explore New Incident File

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Common imports
import os
import timeit
import numpy as np
import pandas as pd
import seaborn as sns
from math import sqrt
from datetime import date
import holidays
sns.set()
import warnings
warnings.filterwarnings("ignore")

# To plot pretty figures
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams["font.family"] = "serif"
mpl.rcParams["font.sans-serif"] = "Verdana"

## VDS Lat Long.csv

In [ ]:
loc_df = pd.read_csv("data/VDS Lat Long.csv", header=0)

# Extract only Kwinana Northbound (inner) main road
loc_df = loc_df[(loc_df['Short Name'].str.contains("KWN-M")) &
      (loc_df['Latitude'] >= -32.091154) & 
      (loc_df['Latitude'] <= -31.96255087)]

loc_df=loc_df.reset_index(drop=True)

# Check minimum and maximum latitude
loc_df.describe()

## Historic_Incidents_2016_-_2019__Points_.csv

In [ ]:
inc_df=pd.read_csv("data/Historic_Incidents_2016_-_2019__Points_.csv",
                   header=0)

# Choose columns
df = inc_df[['OBJECTID','Lat', 'Long', 'WST_Start', 'WST_End',
             'Incident_Type', 'Description', 'Road', 'Location',
             'TrafficCondition']]

# Convert date column to datetimens[64]
df.WST_Start = pd.to_datetime(df.WST_Start)
df.WST_End = pd.to_datetime(df.WST_End)

# Extract only study period
df2 = df[(df.Road == 'Kwinana Fwy') &
  (df.Lat >= -32.091154) &                  # 0080KWN-MU
  (df.Lat <= -31.96323) &                   # Narrow Bridge
  (df.WST_Start >= '2018-01-01 00:00:00') & # between 1 Jan 2018
  (df.WST_End <= '2018-10-25 23:59:00') &   # to 25 Oct 2018
  (df.Description.str.contains('Kwinana Fwy northbound') |
  df.Description.str.contains('Kwinana Fwy both directions'))]

# Split Location column (description) into three new columns
df2[['Condition','Congestion','Caution']] = (df2['Location'].str
                                             .split('[.]\s\s', 2,
                                                    expand=True))

# Choose columns
df2 = df2[['OBJECTID', 'Lat', 'Long', 'WST_Start', 'WST_End',
          'Incident_Type', 'TrafficCondition', 'Congestion']]

# Drop duplicates
df3 = df2.drop_duplicates(subset=['WST_Start', 'WST_End', 'Incident_Type',
                                  'TrafficCondition','Congestion'])

# Round time to nearest minute
df3.WST_Start = df3.WST_Start.round('min')
df3.WST_End = df3.WST_End.round('min')

# Duration = time difference in minute
df3['Duration'] = (df3.WST_End-df3.WST_Start).astype('timedelta64[m]') 
df3.reset_index(drop=True, inplace=True)
df3.info()

In [ ]:
df3.to_csv('data/Kwinana_Fwy_Historic_Incidents_2018_Wide.csv', index=False)

In [ ]:
# Melt dataframe to long format
df3 = pd.melt(df3, id_vars=['OBJECTID', 'Lat', 'Long', 'Incident_Type',
                           'TrafficCondition', 'Congestion', 'Duration'],
              value_name='DateTime').drop('variable', axis=1)

# Fill in data between start and end time
df4 = (df3.groupby('OBJECTID').apply(lambda x: x.drop_duplicates('DateTime')
                              .set_index('DateTime')
                              .resample('1min')
                              .ffill()))
df4.info()

In [ ]:
df4.iloc[:,1:].to_csv('data/Kwinana_Fwy_Historic_Incidents_2018_Long.csv')

### Visualisation

In [ ]:
# Read wide-format incident file
df1 = pd.read_csv('data/Kwinana_Fwy_Historic_Incidents_2018_Wide.csv')

# Drop Duplicate Entry
df1 = df1[df1.Incident_Type != 'Duplicate Entry']
df1 = df1.reset_index(drop=True)

# Convert date column to datetimens[64]
df1.WST_Start = pd.to_datetime(df1.WST_Start)
df1.WST_End = pd.to_datetime(df1.WST_End)

# Replace '/' in Incident_Type column by '/\n'
df1.Incident_Type = df1.Incident_Type.str.replace('/', '/\n')

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ax = sns.barplot(x=df1.Incident_Type.value_counts(),
           y=df1.Incident_Type.value_counts().index)
ax.set_xlabel('Frequency')
ax.set_xlim(right=200)
for p in ax.patches:
  width = p.get_width()
  ax.text(width + 1,
          p.get_y() + p.get_height()/2,
          int(width),
          ha="left",
          va="center")
#plt.savefig('fig/incident_type_1.png', bbox_inches="tight")
plt.show()

- Since 'Special Event', 'Pothole / Road Surface Damage', and 'Hazmat (including spills)' did not affect traffic congestion, these three categories are combined into one category called 'Special Event / Pothole / Hazmat'.

- Flooding and Storm are also combined into one category called 'Flooding / Storm'

In [ ]:
df1.Incident_Type=df1.Incident_Type.replace(['Special Event',
                                            'Pothole /\n Road Surface Damage',
                                            'Hazmat (including spills)'],
                                           'Special Event /\nPothole / Hazmat')

df1.Incident_Type=df1.Incident_Type.replace(['Flooding', 'Storm'],
                                           'Flooding /\nStorm')

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ax = sns.barplot(x=df1.Incident_Type.value_counts(),
           y=df1.Incident_Type.value_counts().index)
ax.set_xlabel('Frequency')
ax.set_xlim(right=200)
for p in ax.patches:
  width = p.get_width()
  ax.text(width + 1,
          p.get_y() + p.get_height()/2,
          int(width),
          ha="left",
          va="center")
#plt.savefig('fig/incident_type_2.png', bbox_inches="tight")
plt.show()

In [ ]:
custom_dict = {'Break Down /\n Tow Away': 7,
               'Debris /\n Trees /\n Lost Loads': 6,
               'Road Crash': 5,
               'Miscellaneous': 4,
               'Animal /\n Livestock': 3,
               'Flooding /\nStorm': 2,
               'Special Event /\nPothole / Hazmat': 1,
               'Vehicle Fire': 0}

df = (df1.groupby(['Incident_Type', 'Congestion'])
 .size().unstack().sort_values(by='Incident_Type',
                               key=lambda x: x.map(custom_dict)))

In [ ]:
temp2 = df1.groupby(['Incident_Type', 'Congestion']).size().unstack()
temp2['sum'] = temp2.sum(axis=1)

In [ ]:
plt.rcParams["figure.figsize"] = (6,6)
(temp2.sort_values(by='sum').iloc[:,:-1]
 .plot(kind='barh', stacked=True, width=0.7))
plt.ylabel('')
plt.xlabel('Frequency')
plt.xlim(right=200)
plt.legend(title='Traffic Condition')
for i, v in enumerate(temp2.sort_values('sum')['sum']):
    plt.text(v+1, i-.15, int(v))
#plt.savefig('fig/incident_congestion_1.png', bbox_inches="tight")
plt.show()

In [ ]:
df_total = df.sum(axis=1)
df_rel = df.div(df_total, 0) * 100

plt.rcParams["figure.figsize"] = (6,6)
df_rel.plot(kind='barh', stacked=True)
plt.ylabel('')
plt.xlabel('Percentage')
plt.legend(loc='lower right')

for n in df_rel: # for each column
    for i, (cs, ab, pc) in enumerate(zip(df_rel.cumsum(1)[n], 
                                         df_rel[n], df[n])):
        if ~np.isnan(cs):
            plt.text(cs - ab / 2, i,
                 str(int(pc)) + '\n(' + str(np.round(ab, 1)) + '%)', 
                 va = 'center', ha = 'center', rotation = 16, fontsize = 8)
#plt.savefig('fig/incident_congestion_2.png', bbox_inches="tight")
plt.show()

In [ ]:
df1.TrafficCondition=df1.TrafficCondition.replace(['Left Emergency Lane Blocked',
                                             'Right Emergency Lane Blocked'],
                                             'Emergency Lane Blocked')

df1.TrafficCondition=df1.TrafficCondition.replace(['Left Lane(s) Blocked',
                                             'Right Lane(s) Blocked',
                                             'Centre Lane(s) Blocked',
                                             'Left Centre Lane(s) Blocked',
                                             'Right Centre Lane(s) Blocked',
                                             'Bus Lane Blocked'],
                                             'Lane(s) Blocked')

df1.TrafficCondition=df1.TrafficCondition.replace(['Left Turning Pocket Blocked',
                                             'Right Turning Pocket Blocked'],
                                             'Turning Pocket Blocked')

In [ ]:
df1.TrafficCondition.value_counts()

In [ ]:
df1.groupby(['TrafficCondition', 'Congestion']).size().unstack()

In [ ]:
df1.groupby(['TrafficCondition', 'Incident_Type']).size().unstack()

In [ ]:
temp1 = df1.groupby(['Incident_Type', 'TrafficCondition']).size().unstack()
temp1['sum'] = temp1.sum(axis=1)

In [ ]:
plt.rcParams["figure.figsize"] = (6,6)
(temp1.sort_values(by='sum').iloc[:,:-1]
 .plot(kind='barh', stacked=True, width=0.7))
plt.ylabel('')
plt.xlabel('Frequency')
plt.xlim(right=200)
plt.legend(title='Traffic Condition')
for i, v in enumerate(temp1.sort_values('sum')['sum']):
    plt.text(v+1, i-.15, int(v))
plt.savefig('fig/incident_condition_1.png', bbox_inches="tight")
plt.show()

In [ ]:
df = (df1.groupby(['Incident_Type', 'TrafficCondition'])
 .size().unstack().sort_values(by='Incident_Type',
                               key=lambda x: x.map(custom_dict)))

df_total = df.sum(axis=1)
df_rel = df.div(df_total, 0) * 100

plt.rcParams["figure.figsize"] = (6,6)
df_rel.plot(kind='barh', stacked=True)
plt.ylabel('')
plt.xlabel('Percentage')

plt.legend(loc='lower left', bbox_to_anchor=(0.55, 0.001))

for n in df_rel: # for each column
    for i, (cs, ab, pc) in enumerate(zip(df_rel.cumsum(1)[n], 
                                         df_rel[n], df[n])):
        if ~np.isnan(cs):
            plt.text((cs - ab / 2)+1.2, i,
                 str(int(pc)) + '\n(' + str(np.round(ab, 1)) + '%)', 
                 va = 'center', ha = 'center', rotation = 25, fontsize = 8)
#plt.savefig('fig/incident_condition_2.png', bbox_inches="tight")
plt.show()

In [ ]:
def func2(a):
    if -32.091154 <= a < -32.080696:
        return "1"
    elif -32.080696 <= a < -32.074042:
        return "2"
    elif -32.074042 <= a < -32.071075:
        return "3"
    elif -32.071075 <= a < -32.057092:
        return "4"
    elif -32.057092 <= a < -32.052286:
        return "5"
    elif -32.052286 <= a < -32.043637:
        return "6"
    elif -32.043637 <= a < -32.040758:
        return "7"
    elif -32.040758 <= a < -32.030254:
        return "8"
    elif -32.030254 <= a < -32.012242:
        return "9"
    elif -32.012242 <= a < -32.010690:
        return "10"
    elif -32.010690 <= a < -32.003147:
        return "11"
    elif -32.003147 <= a < -31.969905:
        return "12"
    elif -31.969905 <= a < -31.966753:
        return "13"
    elif a >= -31.966753 :
        return "14"
    else:
        return "Other"

df1['ID'] = df1['Lat'].apply(lambda x: func2(x))
df1.head()

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ax = sns.barplot(x=df1.ID.astype(str).value_counts(),
           y=df1.ID.astype(str).value_counts().index)
ax.set_xlabel('Frequency')
ax.set_ylabel('Link')
ax.set_xlim(right=85)
for p in ax.patches:
  width = p.get_width()
  ax.text(width + 1,
          p.get_y() + p.get_height()/2,
          int(width),
          ha="left",
          va="center")
#plt.savefig('fig/incident_link_1.png', bbox_inches="tight")
plt.show()

In [ ]:
temp = df1.groupby(['ID', 'Incident_Type']).size().unstack()
temp['sum'] = temp.sum(axis=1)

In [ ]:
plt.rcParams["figure.figsize"] = (6,6)
(temp.sort_values(by='sum').iloc[:,:-1]
 .plot(kind='barh', stacked=True, width=0.7))
plt.ylabel('Link')
plt.xlabel('Frequency')
plt.xlim(right=85)
plt.legend(title='Incident Type', loc='lower left',
           bbox_to_anchor=(0.61, -0.012))
for i, v in enumerate(temp.sort_values('sum')['sum']):
    plt.text(v+1, i-.15, int(v))
#plt.savefig('fig/incident_link_2.png', bbox_inches="tight")
plt.show()

In [ ]:
del temp, temp1, temp2

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors

def colorbar_index(ncolors, cmap):
    cmap = cmap_discretize(cmap, ncolors)
    mappable = cm.ScalarMappable(cmap=cmap)
    mappable.set_array([])
    mappable.set_clim(-0.5, ncolors+0.5)
    colorbar = plt.colorbar(mappable)
    colorbar.set_ticks(np.linspace(0, ncolors, ncolors))
    colorbar.set_ticklabels(range(1, ncolors+1))
    colorbar.set_label('Link')
    
def cmap_discretize(cmap, N):
    """Return a discrete colormap from the continuous colormap cmap.

        cmap: colormap instance, eg. cm.jet. 
        N: number of colors.

    Example
        x = resize(arange(100), (5,100))
        djet = cmap_discretize(cm.jet, 5)
        imshow(x, cmap=djet)
    """

    if type(cmap) == str:
        cmap = plt.get_cmap(cmap)
    colors_i = np.concatenate((np.linspace(0, 1., N), (0.,0.,0.,0.)))
    colors_rgba = cmap(colors_i)
    indices = np.linspace(0, 1., N+1)
    cdict = {}
    for ki,key in enumerate(('red','green','blue')):
        cdict[key] = [ (indices[i], colors_rgba[i-1,ki], colors_rgba[i,ki])
                       for i in range(N+1) ]
    # Return colormap object.
    return mcolors.LinearSegmentedColormap(cmap.name + "_%d"%N, cdict, 1024)


cmp = mpl.colors.ListedColormap(['#ebac23', '#b80058', '#008cf9',
                                 '#006e00', '#00bbad', '#d163e6',
                                 '#b24502', '#ff9287', '#5954d6',
                                 '#00c6f8', '#878500', '#00a76c',
                                 '#bdbdbd', '#000078', '#b51d14'])
df1.plot(kind='scatter', x='Long', y='Lat', alpha=0.5,
        s='Duration', c='ID', label='Duration',
        cmap=cmp, colorbar=False, rot=45)
plt.ticklabel_format(useOffset=False)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(markerscale=0.2)
colorbar_index(ncolors=14, cmap=cmp)
#plt.savefig('fig/incident_loc_link_2.png', bbox_inches="tight")
plt.show()

In [ ]:
# cmp = plt.get_cmap('prism')
# df1.plot(kind='scatter', x='Long', y='Lat', alpha=0.5,
#         s='Duration', c='ID', label='Duration',
#         cmap=cmp, colorbar=False, rot=45)
# plt.ticklabel_format(useOffset=False)
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')
# plt.legend(markerscale=0.2)
# colorbar_index(ncolors=14, cmap=cmp)
#plt.savefig('fig/incident_loc_link_1.png', bbox_inches="tight")
# plt.show()

## Merge Traffic data with new incident file

In [ ]:
# Load long format incident file
df2 = pd.read_csv('data/Kwinana_Fwy_Historic_Incidents_2018_Long.csv')

# Remove Long, Congestion, and Duration
df2 = df2[['DateTime', 'Lat', 'Incident_Type',
           'TrafficCondition', 'Congestion']]

df2 = df2[df2.Incident_Type != 'Duplicate Entry']
df2.DateTime = pd.to_datetime(df2.DateTime)
df2.Incident_Type = df2.Incident_Type.str.replace('/', '/\n')

# Replace columns' values
df2.Incident_Type=df2.Incident_Type.replace(['Special Event',
                                            'Pothole /\n Road Surface Damage',
                                            'Hazmat (including spills)'],
                                           'Special Event /\nPothole / Hazmat')

df2.Incident_Type=df2.Incident_Type.replace(['Flooding', 'Storm'],
                                           'Flooding /\nStorm')

df2.TrafficCondition=df2.TrafficCondition.replace(['Left Emergency Lane Blocked',
                                             'Right Emergency Lane Blocked'],
                                             'Emergency Lane Blocked')

df2.TrafficCondition=df2.TrafficCondition.replace(['Left Lane(s) Blocked',
                                             'Right Lane(s) Blocked',
                                             'Centre Lane(s) Blocked',
                                             'Left Centre Lane(s) Blocked',
                                             'Right Centre Lane(s) Blocked',
                                             'Bus Lane Blocked'],
                                             'Lane(s) Blocked')

df2.TrafficCondition=df2.TrafficCondition.replace(['Left Turning Pocket Blocked',
                                             'Right Turning Pocket Blocked'],
                                             'Turning Pocket Blocked')

# Add ID column based on latitude
def func2(a):
    if -32.091154 <= a < -32.080696:
        return "1"
    elif -32.080696 <= a < -32.074042:
        return "2"
    elif -32.074042 <= a < -32.071075:
        return "3"
    elif -32.071075 <= a < -32.057092:
        return "4"
    elif -32.057092 <= a < -32.052286:
        return "5"
    elif -32.052286 <= a < -32.043637:
        return "6"
    elif -32.043637 <= a < -32.040758:
        return "7"
    elif -32.040758 <= a < -32.030254:
        return "8"
    elif -32.030254 <= a < -32.012242:
        return "9"
    elif -32.012242 <= a < -32.010690:
        return "10"
    elif -32.010690 <= a < -32.003147:
        return "11"
    elif -32.003147 <= a < -31.969905:
        return "12"
    elif -31.969905 <= a < -31.966753:
        return "13"
    elif a >= -31.966753 :
        return "14"
    else:
        return "Other"

df2['ID'] = df2['Lat'].apply(lambda x: func2(x))
df2.ID = df2.ID.astype('int64')

# Rearrange column
df2 = df2[['ID', 'DateTime', 'Incident_Type',
           'TrafficCondition', 'Congestion']]



In [ ]:
# Get the minimum and maximum latitude of each ID

# Add ID column based on detector number
def func(a):
    if "0220" in a or "0210" in a:
        return "1"
    elif "0091" in a or "0200" in a:
        return "2"
    elif "0090" in a:
        return "3"
    elif "0190" in a or "0089" in a:
        return "4"
    elif "0180" in a or "0700" in a:
        return "5"
    elif "0170" in a or "0702" in a:
        return "6"
    elif "0160" in a:
        return "7"
    elif "0088" in a or "0150" in a:
        return "8"
    elif "0087" in a or "0140" in a or "0086" in a:
        return "9"
    elif "0130" in a:
        return "10"
    elif "0085" in a or "0084" in a:
        return "11"
    elif "0120" in a or "0003" in a or "0083" in a or "0100" in a or "0082" in a or "0081" in a:
        return "12"
    elif "0002" in a:
        return "13"
    elif "0080" in a:
        return "14"
    else:
        return "Other"
    
loc_df['ID'] = loc_df['Short Name'].apply(lambda x: func(x))

minloc=loc_df.groupby('ID').Latitude.describe()[['min','max']].reset_index()
minloc.ID = minloc.ID.astype(int)
minloc=minloc.sort_values(by='ID')

In [ ]:
# Check duplicate entries. (>1 incident occurred at the same time)
df2[df2.duplicated(subset=['ID', 'DateTime'], keep=False)]

In [ ]:
# Load traffic data file
df3 = pd.read_csv('data/LAD.csv')
df3 = df3[['ID', 'DateTime', 'Length', 'Volume', 'Speed', 'Occupancy']]
df3.DateTime = pd.to_datetime(df3.DateTime)

In [ ]:
# Merge data file (duplicate incidents still there)
df_merged = (pd.merge(df3.sort_values(['ID', 'DateTime']),
                      df2.sort_values(['ID', 'DateTime']),
                      how='left',
                      on=['ID', 'DateTime'])
             .sort_values(['ID', 'DateTime'])
             .reset_index(drop=True)
            )

In [ ]:
# Duplicate entries
# df_merged[((df_merged.ID==12) & (df_merged.DateTime == '2018-01-23 13:38:00')) | 
#          ((df_merged.ID==12) & (df_merged.DateTime == '2018-05-25 10:47:00')) |
#          ((df_merged.ID==12) & (df_merged.DateTime == '2018-06-06 11:56:00')) |
#          ((df_merged.ID==12) & (df_merged.DateTime == '2018-06-20 12:20:00')) |
#          ((df_merged.ID==14) & (df_merged.DateTime == '2018-07-22 06:55:00')) |
#          ((df_merged.ID==9) & (df_merged.DateTime == '2018-09-20 10:19:00')) |
#          ((df_merged.ID==1) & (df_merged.DateTime == '2018-09-25 06:34:00')) |
#          ((df_merged.ID==14) & (df_merged.DateTime == '2018-10-03 18:06:00')) |
#          ((df_merged.ID==11) & (df_merged.DateTime == '2018-10-08 08:46:00'))]

In [ ]:
# Deal with duplicate entries by combining them into one column seperated by ', '
df_merged2 = (df_merged.set_index(['ID', 'DateTime', 'Length',
                                'Volume', 'Speed', 'Occupancy'])
              .astype(str)
              .groupby(['ID', 'DateTime', 'Length',
                                'Volume', 'Speed', 'Occupancy'])
              .agg(', '.join)
              .reset_index())

In [ ]:
# Check duplicate entries
# df_merged2[((df_merged2.ID==12) & (df_merged2.DateTime == '2018-01-23 13:38:00'))]

In [ ]:
df_merged2.Incident_Type.value_counts()

In [ ]:
df_merged2.TrafficCondition.value_counts()

In [ ]:
df_merged2.TrafficCondition=df_merged2.TrafficCondition.replace({
    'All Lanes Open, Emergency Lane Blocked': 'Emergency Lane Blocked'
})

In [ ]:
df_merged2.to_csv('data/LAD+incident2.csv')